 # projeto sem objetivo pre definido

* primeiro fazer coleta dos dados para armazenamento 
*  sera feito coleta atraves de web scraping
* site: https://www.zapimoveis.com.br
* limpeza de dados
* talvez crie um banco de dados para mostrar a criacao de um banco de dados
* talvez pode ser feito uma coleta diaria durante um tempo para saber qto tempo um imovel fica disponivel para venda, qual o tempo medio de espera para ser vendido, qual o preço medio dos  imoveis mais vendidos 

apos coleta sera feita uma analise exploratoria para saber o que sera possivel de analisar, como por exemplo, valor media das casa, valor medio por regiao, a media de comodos, regioes  e comodos afetam os preços, qual tem mais impacto...?

tecnologia esperadas q serao usadas

* python
    * selenium ou beutiful soup para coleta de dados
    * pandas para exploraçao e limpeza

* power bi
    * para apresentaçao final

*github para fazer os historicos de evoluçao diaria e ter noçao de qto tempo levou para fazer o projeto (nao estou me obrigando a ter prazo, tenho outro trabalho entao sera feito nas horas disponiveis sem afetar outros compromissos)






### Importando as bibliotecas necessarias

In [1]:
import pandas as pd
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from datetime import date

foi utiliza a biblioteca de webscrapping Selinum, pois axiste alguns link q devem ser clicado e o Selenium, faz isso com excelencia

Foi feito um webscrapping na pagina: https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=1&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda (que vou me referir a ela como pagina base), pois ja considera alguns filtros de entrada para q seja mais rapida a localizaçao dos elementos pre estabelicido da busca. Esses elementos sao: casa, venda e em ordem crescente de valores

o web scrapping foi feito com abertura da janela do navegador para q houvesse um controle melhor dos erros e acertos, e com a aba maximizada para ter um controle melhor sobre a janela aberta e nao correr o risco de a janela ser interpretada com css mobile

esse foi o codigo inicial para explorar e iniciar os teste de scrapping

~~~Python

#comando para abrir a janela do navegador
driver_casa = webdriver.Chrome()    

#pagina escolhida para iniciar o scrapping
driver_casa.get('https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=1&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda')

#para maximizar a tela do navegador
driver_casa.maximize_window()

#para verificar em qual pagina eu estou 
original_window = driver_casa.current_window_handle

#pegando os anuncios das casas
anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

#conferindo a quantidade de elementos 
len(anuncios)
~~~

explorando um pouco a pgina base, percebi q era possivel extrair a maior parte dos dados para estudos, porem resolvi entrar em cada anuncio, para poder tanto melhorar meu aprendizado na biblioteca Selenium, quanto para pegar dados que haviam apenas dentro dos anuncios, como por exemplo o id do anuncio, que sera usado mais pra frente para fazer filtragem para novas buscas

uma vez que tive acesso a pagina do anuncio peguei os dados:

    > valor do imovel
    
    > valor do condominio
    
    > valor do iptu
    
    > comodos:
        * area 
        * quantidade de banheiros
        * quantidade de vagas de garagem
        * quantidade de quartos
       
    > endereço
    
    > descriçoes (para possiveis informaçoes complementares)
    
    > observaçoes (para possiveis informaçoes complementares)    

usei essa parte do codigo para explorar as abas q estava abertas e saber se haviam mais do que uma, pois dependendo de qual aba eu estivesse nao seria possivel fazer acoes em outro aba, entao teria q fazer a troca de aba pelo codigo q o "windo.handle" nos da

~~~Python

#window_handles eh uma lista de todas as guias q estao abertas, usar esse for pra saber quais os nomes das guias
for window_handle in driver_casa.window_handles:
print(window_handle)

# pagina_atual foi usado para confirmar a pagina atual
pagina_atual = driver_casa.current_window_handle

#url foi utilizada para poder, tanto pegar a url dentro dos anuncios
url = driver_casa.current_url
~~~

# Codigo do projeto

In [ ]:
dados = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Lista Comodos','Lista Caracteristicas', 'Obserevacoes'])

# for para avancar a paginas base (paginas q contem os anuncios)
for indice in range(42,415): 

    # abrir a janela do navegador
    driver_casa = webdriver.Chrome() 

    # url = pagina base, a pagina base é determinada pelo valor do indice do for acima
    url = f'https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina={indice}&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda' 

    # busca e carrega a pagina q esta na url
    driver_casa.get(url)

    #maximizar a janela para ter sempre o mesmo modelo de busca de css
    driver_casa.maximize_window()

    # grava qual o cadigo padrao da pagina base
    original_window = driver_casa.current_window_handle

    # pega uma lista com o elementos dos anuncios
    anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

    #for para entrar em cada anuncio
    for i in range(len(anuncios)):
        # todos os try: except: sao para nao travar o codigo durante a colheita de dados dos anuncio, carro de erro na pagina ou alguma outra eventualidade
        try:
            while len(driver_casa.window_handles) == 1: # while serve para corfirmar a entrada nos anuncios
                try:
                    sleep(1)
                    driver_casa.find_elements(By.CLASS_NAME, 'card-container')[i].click() 
                except:
                    print('erro no click do anuncio')
                if len(driver_casa.window_handles) > 1 : #caso if True, troca-se para pagina do anuncio
                    driver_casa.switch_to.window(driver_casa.window_handles[1])
                    break



            endereco = None
            preco = None
            condominio = None
            iptu = None
            comodos = None
            lista_comodos = None
            caracteristicas = None
            lista_caracteristicas = None
            url_id_anuncio = None

            sleep(1.5)


            #pega a url para o dataset para ser transformada em id
            try:
                url_id_anuncio = driver_casa.current_url
            except:
                print('erro url_id_anuncio =====================')

            #pega string endereço para DataSet
            try:
                endereco= driver_casa.find_element(By.CLASS_NAME, 'info__map-link ') 
            except:
                print('erro endereco =====================')


            #pega string valor dos preço para DataSet
            try:
                preco = driver_casa.find_element(By.CLASS_NAME, 'price__item--main')
            except:
                print('erro preco =====================')


            #pega string valor do condominio para o DataSet
            try:
                condominio = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[0]
            except:
                print('erro condominio =====================')


            #pega string valor do iptu para o DataSet
            try:
                iptu = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[1]
            except:
                print('erro iptu =====================')


            #pega as observaçoes para colocar no dataset para consultas complemetares
            try: 
                obserevacoes = driver_casa.find_element(By.CLASS_NAME, 'amenities__base-description')
            except:
                print('erro obserevacoes =====================')


            #pega os elementos e tranforma em uma lista dos caracteristicas da casa, para eventual consulta complementar (vai para o dataset)
            try:
                caracteristicas = driver_casa.find_elements(By.CLASS_NAME, 'amenities__list-item')
                lista_caracteristicas = [caracteristica.text for caracteristica in caracteristicas]
            except:
                print('erro caracteristicas =====================')


            #pega os elementos e tranforma em uma lista dos comodos da casa (vai para o dataset)
            try:
                comodos = driver_casa.find_elements(By.CLASS_NAME, 'feature__item')
                lista_comodos = [comodo.text for comodo in comodos]   
            except:
                print('erro comodos =====================')

            dic_dados = {'Endereço':endereco.text,
                     'Preço' : preco.text,
                     'Condominio': condominio.text,
                     'Iptu': iptu.text,
                     'Lista Comodos': lista_comodos,
                     'Lista Caracteristicas' : lista_caracteristicas,
                     'Obserevacoes' : obserevacoes.text,
                     'url_id_anuncio' : url_id_anuncio}

            while len(driver_casa.window_handles) > 1:
                driver_casa.switch_to.window(driver_casa.window_handles[1])
                driver_casa.close()
                driver_casa.switch_to.window(original_window)

            dados = dados.append(dic_dados, ignore_index = True)
            print('dados adicionado!!')
        except:
            print('************ FALHA de CARREGAR anuncio!!!!!!!!!!!!!!!!')
    driver_casa.quit()
    data_atual = date.today()
    data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
    tabela_imoveis_raw = dados.to_csv(f'tabela_imoveis_raw2_{data_atual}.csv', sep = ',', index = False)
dados

C:\Users\jpnun\AppData\Local\Temp\ipykernel_3772\3403427635.py:123: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados = dados.append(dic_dados, ignore_index = True)


dados adicionado!!


C:\Users\jpnun\AppData\Local\Temp\ipykernel_3772\3403427635.py:123: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados = dados.append(dic_dados, ignore_index = True)


dados adicionado!!


C:\Users\jpnun\AppData\Local\Temp\ipykernel_3772\3403427635.py:123: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados = dados.append(dic_dados, ignore_index = True)


...

dados adicionado!!


C:\Users\jpnun\AppData\Local\Temp\ipykernel_3772\3403427635.py:123: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados = dados.append(dic_dados, ignore_index = True)


dados adicionado!!


C:\Users\jpnun\AppData\Local\Temp\ipykernel_3772\3403427635.py:123: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados = dados.append(dic_dados, ignore_index = True)


dados adicionado!!


In [62]:
dados['Lista Comodos']

0     [, , , , 120 m², 2 quartos, 2 vagas, 1 banheiro]
1    [, , , , 138 m², 3 quartos, 2 vagas, 2 banheiros]
Name: Lista Comodos, dtype: object

In [21]:
type(dados['Lista Caracteristicas'][0])

list

In [64]:
data_atual = date.today()
data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
tabela_imoveis_raw = dados.to_csv(f'tabela_imoveis_raw_{data_atual}.csv', sep = ',', index = False)


In [35]:
dados.head(1)

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"Rua Coronel Manoel Martins Júnior, 0 - Jardim ...",R$ 42.000,condomínio não informado,IPTU não informado,"[, , , , 120 m², 2 quartos, 2 vagas, 1 banheiro]",[],A casa está localizado no bairro Jardim Esplan...,https://www.zapimoveis.com.br/imovel/venda-cas...


In [66]:
df = pd.read_csv('tabela_imoveis_raw2_15_3_2022.csv')
df.head()

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"Rua Coronel Manoel Martins Júnior, 0 - Jardim ...",R$ 42.000,condomínio não informado,IPTU não informado,"['', '', '', '', '120 m²', '2 quartos', '2 vag...",[],A casa está localizado no bairro Jardim Esplan...,https://www.zapimoveis.com.br/imovel/venda-cas...
1,"Rua Bariloche, 0 - Jardim América, São José do...",R$ 43.000,condomínio não informado,IPTU não informado,"['', '', '', '', '138 m²', '3 quartos', '2 vag...","['Aceita animais', 'Ar-condicionado', 'Churras...",A casa está localizado no bairro Jardim Améric...,https://www.zapimoveis.com.br/imovel/venda-cas...


In [67]:
df.shape

(2, 8)

In [42]:
dados['Lista Caracteristicas'].apply(lambda x:  None if x == [] else x)

0                                                None
1    [Aceita animais, Ar-condicionado, Churrasqueira]
Name: Lista Caracteristicas, dtype: object

In [44]:
data_atual = date.today()
data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
tabela_imoveis_raw = dados.to_csv(f'tabela_imoveis2_raw_{data_atual}.csv', sep = ',', index = False)

In [27]:
type(df['Lista Caracteristicas'][0])

str

In [45]:
df = pd.read_csv('tabela_imoveis_raw2_14_3_2022.csv')
df.head(1)

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"Rua Coronel Manoel Martins Júnior, 0 - Jardim ...",R$ 42.000,condomínio não informado,IPTU não informado,"['', '', '', '', '120 m²', '2 quartos', '2 vag...",[],A casa está localizado no bairro Jardim Esplan...,https://www.zapimoveis.com.br/imovel/venda-cas...


In [57]:
#df['Lista Caracteristicas'] = df.values.tolist()
type(df['Lista Caracteristicas'][1])

list